In [4]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests
from functionssb import *

In [5]:
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [6]:
current_week = fetch_max_gw(gs_connection, gameweek_results_table, prod_google_sheet_key)

Error: Worksheet not found: gameweek_results_2


In [4]:
#make this cache indefintely and reset on update
gameweek_df = fetch_gameweek_data(
        gs_connection,
        gameweek_results_table,
        prod_google_sheet_key,
        ["event", "points", "total_points", "event_transfers_cost", "points_on_bench"],)

In [5]:
#make this cache indefintely and reset on update
all_managers = fetch_manager_data(gs_connection, managers_table, prod_google_sheet_key, [])

In [6]:
#make this cache indefintely and reset on update
all_managers = fetch_manager_data(gs_connection, managers_table, prod_google_sheet_key, [])

# Filter active managers and sort their player names
active_managers = all_managers[all_managers.active == 'Yes']["player_name"].tolist()
active_managers.insert(0, "")
active_managers.sort()

# Sort all managers' player names
managers = all_managers["player_name"].tolist()
managers.insert(0, "")
managers.sort()


#this has a cache reset every 6 hours because people can nominate or uno reverse and the change should be reflected
drinks = fetch_drinks_data(gs_connection, drinks_table, prod_google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])
drinks_display = build_drinks_display(drinks, current_week)
drinks_display.index = np.arange(1, len(drinks_display) + 1)

drinks_display_expanded = build_drinks_display_expanded(drinks)
drinks_display_expanded.index = np.arange(1, len(drinks_display_expanded) + 1)

litres, missed_pen_count, red_card_count, own_goal_count, nomination_count = analyze_drinks(drinks)
first_place, last_place, first_team_name = get_first_last(gameweek_df, current_week, drinks)

/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drinks_display["formatted_deadline_date"] = drinks_display[
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drinks_display.rename(


In [9]:
df, current_gw, drinks = gameweek_df, current_week, drinks

In [13]:
not_completed_df = drinks[
        (
            (drinks["nomination_completed_date"] == "Not Completed")
            & ((drinks["event"] < current_gw))
        )
    ][["drinker_name", "nomination_deadline_date"]]
not_completed_df

,drinker_name,nomination_deadline_date
31,Devon Jansen,1693864799
49,Rhys Floyd,1695765599
55,Devon Jansen,1696283999
63,Ryan Shackleton,1697061599
72,Devon Jansen,1697061599
81,Rhys Floyd,1697023650
90,Rhys Floyd,1699307999
91,Jonno Frame,1699307999
94,Ryan Shackleton,1699999199
101,Devlin Sharp,1699962201


In [19]:
current_gw_df2 = current_gw_df[~current_gw_df["player_name"].isin(not_completed_df['drinker_name'])]

In [20]:
current_gw_df2

,event,entry,player_name,entry_name,points,total_points,event_transfers,event_transfers_cost,points_on_bench
415,20,6394380,Divyam Dixit,To be Coutinho'd,80,1129,2,0,6
418,20,4719647,Jason Perry,Flekken The Bean,76,1085,0,0,3
419,20,247866,Connor McDonald,AllHitsNoMisses,85,1184,0,0,10
421,20,2429570,Hethe Brinkman,Hairy Man-Gina,63,1074,0,0,1
422,20,3854998,Cole Floyd,ChickenTikkaMoSalah,75,1087,2,0,8
424,20,574473,Liam Smorfitt,Lik Matip,84,1051,0,0,0
425,20,3827624,Lawrence Fowle,VictoriousSecret,37,950,1,0,1
426,20,1805743,Peter Wertz,Chicken Mosalah,62,1001,0,0,1
427,20,6944109,Luca Milanesi,Childish Firmino,86,1091,0,0,4
428,20,1143100,Daniel van den Berg,LongShot FC,83,1153,0,0,3


In [ ]:
if drinks is None or len(drinks) == 0:
        return None, None, None  # Handle missing drinks DataFrame

    current_gw_df = df[df["event"] == current_gw]
    current_unix_time = int(time.time())

    # Correct the logical condition grouping
    not_completed_df = drinks[
        (
            (drinks["nomination_completed_date"] == "Not Completed")
            & ((drinks["event"] < current_gw))
        )
    ][["drinker_name", "nomination_deadline_date"]]

    not_completed_df = not_completed_df[
        not_completed_df["nomination_deadline_date"].astype(int) < current_unix_time
    ]

    if len(current_gw_df) == 0:
        return None, None, None  # GW not finished

    current_gw_df = current_gw_df[~current_gw_df["player_name"].isin(not_completed_df)]

    if len(current_gw_df) == 0:
        return None, None, None  # No valid entries after exclusion

    sorted_df = current_gw_df.sort_values(
        by=["points", "total_points", "player_name"], ascending=[False, False, True]
    )

    # Safely access first and last place names
    first_place_name = sorted_df.iloc[0].player_name if len(sorted_df) > 0 else None
    last_place_name = sorted_df.iloc[-1].player_name if len(sorted_df) > 0 else None
    first_team_name = sorted_df.iloc[0].entry_name if len(sorted_df) > 0 else None

In [4]:

#update functions
manager_details = managers_update(gs_connection, league_endpoint, current_gw)
gameweek_results_update(gs_connection, current_gw, manager_details)
auto_assign_drinks(gs_connection, gameweek_results_table, gameweek_teams_table, prod_google_sheet_key, current_gw)
st.cache_data.clear()         

2023-12-30 16:15:58.721 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
drinks = fetch_drinks_data(gs_connection, drinks_table, prod_google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])

In [4]:
drinks

,event,nominator_name,drinker_name,drink_type,nomination_created_date,nomination_deadline_date,nomination_completed_date,quantity,drink_size,start_time,end_time
0,1,auto,Jonno Frame,last place,1692070784,1692741599,1692725340,,330.0,4.25,12.04
1,1,Devlin Sharp,Connor McDonald,nomination,1692081950,1692741599,1692110760,,330.0,5.38,11.04
2,1,Devlin Sharp,Tijil Jugadoe,nomination,1692081950,1692741599,1692289560,,330.0,0.00,10.88
3,1,Devlin Sharp,Niklas Wietzorrek,nomination,1692081950,1692741599,1692462600,,330.0,1.03,7.38
4,2,auto,Tijil Jugadoe,red card,1692694965,1693346399,1692688020,,440.0,0.00,8.70
...,...,...,...,...,...,...,...,...,...,...,...
132,17,Connor McDonald,Hethe Brinkman,nomination,1703152770,1703764770,Not Completed,,NaN,NaN,NaN
133,18,auto,Jonno Frame,last place,1703482751,1704146399,Not Completed,,NaN,NaN,NaN
134,18,Marco Gouveia,Marco Gouveia,uno reverse,1703507611,1704119611,Not Completed,,NaN,NaN,NaN
135,18,Marco Gouveia,Tijil Jugadoe,nomination,1703507611,1704119611,1703612181,,500.0,NaN,NaN


In [7]:
df = drinks
df["quantity"] = 1
df = df.iloc[:, [2, 5, 6, 7]]
df.rename(columns={"drinker_name": "Name"}, inplace=True)
df


/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_1456/2993913015.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"drinker_name": "Name"}, inplace=True)


,Name,nomination_deadline_date,nomination_completed_date,quantity
0,Jonno Frame,1692748799,1692725340,1
1,Connor McDonald,1692748799,1692110760,1
2,Tijil Jugadoe,1692748799,1692289560,1
3,Niklas Wietzorrek,1692748799,1692462600,1
4,Tijil Jugadoe,1693353599,1692688020,1
5,Rhys Floyd,1693353599,1693330440,1
6,Lawrence Fowle,1693353599,1692960660,1
7,Divyam Dixit,1693353599,1693319340,1
8,Alex Wietzorrek,1693353599,1692806460,1
9,Jason Perry,1693353599,1692688260,1


In [8]:
time1 = 'Not Completed'
time2 = '1693864799'

In [11]:
x = int(time2)
date_obj = datetime.fromtimestamp(x)
date_obj.strftime('%d %b')

'04 Sep'

In [ ]:
try:
    timestamp = int(date_str)  # Assuming date_str is a Unix timestamp in string format
    date_obj = datetime.fromtimestamp(timestamp)
    return date_obj.strftime('%d %b')
except ValueError:
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%y %H:%M')
        return date_obj.strftime('%d %b')
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%y')
            return date_obj.strftime('%d %b')
        except ValueError:
            return "Not Completed"

In [ ]:
def create_unix(date_str):
    try:
        timestamp = int(date_str)
        return timestamp
    except ValueError:
        return "Not Completed"

In [14]:
df["nomination_completed_date"] = df["nomination_completed_date"].apply(lambda x: format_date(x))
df["nomination_deadline_date"] = df["nomination_deadline_date"].apply(lambda x: format_date(x))
df

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_51732/3995295841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_completed_date"] = df["nomination_completed_date"].apply(lambda x: format_date(x))
/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_51732/3995295841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_deadline_date"] = df["nomination_deadline_date"].apply(lambda x: format_date(x))


,Name,nomination_deadline_date,nomination_completed_date,quantity
0,Jonno Frame,Not Completed,Not Completed,1
1,Connor McDonald,Not Completed,Not Completed,1
2,Tijil Jugadoe,Not Completed,Not Completed,1
3,Niklas Wietzorrek,Not Completed,Not Completed,1
4,Tijil Jugadoe,Not Completed,Not Completed,1
5,Rhys Floyd,Not Completed,Not Completed,1
6,Lawrence Fowle,Not Completed,Not Completed,1
7,Divyam Dixit,Not Completed,Not Completed,1
8,Alex Wietzorrek,Not Completed,Not Completed,1
9,Jason Perry,Not Completed,Not Completed,1


In [5]:
df = categories(drinks)
df

/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"drinker_name": "Name"}, inplace=True)
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:510: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_completed_date"] = pd.to_datetime(df["nomination_completed_date"], errors="coerce")
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Name,nomination_deadline_date,nomination_completed_date,quantity,Category
0,Jonno Frame,2023-08-22 23:59:00,2023-08-22 19:29:00,1,Completed
1,Connor McDonald,2023-08-22 23:59:00,2023-08-15 16:46:00,1,Completed
2,Tijil Jugadoe,2023-08-22 23:59:00,2023-08-17 18:26:00,1,Completed
3,Niklas Wietzorrek,2023-08-22 23:59:00,2023-08-19 18:30:00,1,Completed
4,Tijil Jugadoe,2023-08-29 23:59:00,2023-08-22 09:07:00,1,Completed
5,Rhys Floyd,2023-08-29 23:59:00,NaT,1,Outstanding
6,Lawrence Fowle,2023-08-29 23:59:00,2023-08-25 12:51:00,1,Completed
7,Divyam Dixit,2023-08-29 23:59:00,2023-08-29 16:29:00,1,Completed
8,Alex Wietzorrek,2023-08-29 23:59:00,2023-08-23 18:01:00,1,Completed
9,Jason Perry,2023-08-29 23:59:00,2023-08-22 09:11:00,1,Completed


In [6]:
df.dtypes

Name                                 object
nomination_deadline_date     datetime64[ns]
nomination_completed_date    datetime64[ns]
quantity                              int64
Category                             object
dtype: object

In [7]:
a = df.iloc[33]["nomination_deadline_date"]

In [8]:
b = df.iloc[33]["nomination_completed_date"]

In [9]:
a>b

True